In [3]:
from bs4 import BeautifulSoup
import requests,re
import csv
import pandas as pd
import numpy as np

In [4]:
url = 'https://www.propertypro.ng/property-for-rent/in/oyo/ibadan?search=&auto=&type=&bedroom=&min_price=&max_price='

In [5]:
s = requests.Session()
r = s.get(url,timeout=10)
r

<Response [200]>

In [6]:
c = r.content

In [8]:
soup = BeautifulSoup(c, 'html.parser')

In [167]:
scraped=[]

base_url='https://www.propertypro.ng/property-for-rent/in/oyo/ibadan?search=&auto=&type=&bedroom=&min_price=&max_price=&page='

for page in range(1,74):
    print(base_url+str(page))
    r= requests.get(base_url+str(page))
    c= r.content
    soup= BeautifulSoup(c,"html.parser")
        
    classes = ["single-room-sale listings-property"]
    for class_ in classes:
        real=soup.find_all("div",{"class":class_})

        for i in list(range(0,len(real))):
            d={}
            d["page" ] = page
            try:
                d["title"]=real[i].find('h3', {'class':'listings-property-title2'}).get_text().strip()
            except (IndexError,TypeError,AttributeError) as e:
                d["title"]= None
            try:
                d["description"]=real[i].find('p', {'class':'d-none d-sm-block'}).get_text().strip()
            except (IndexError,TypeError,AttributeError) as e:
                d["description"]= None  
            try:
                d["condition"]=real[i].find('div', {'class':'furnished-btn'}).get_text().strip()
            except (IndexError,TypeError,AttributeError) as e:
                d["condition"]= None
            try:
                d["rooms"]=real[i].find('div', {'class':'fur-areea'}).get_text().strip()
            except (IndexError,TypeError,AttributeError) as e:
                d["rooms"]= None    
            try:
                d["address"]= re.findall(r'(.*?)Ibadan',real[i].find('h4', None).get_text())
            except (IndexError,TypeError,AttributeError) as e:
                d["address"]= None
            try:
                d["price"]=real[i].find('h3', {'class':'listings-price'}).get_text().strip()
            except (IndexError,TypeError,AttributeError) as e:
                d["price"]= None  
            scraped.append(d)
            #print(l)

https://www.privateproperty.com.ng/property-for-rent?search=Ibadan+%2C+Oyo&auto=&ptype=&bedroom=&min_price=&max_price=1
https://www.privateproperty.com.ng/property-for-rent?search=Ibadan+%2C+Oyo&auto=&ptype=&bedroom=&min_price=&max_price=2


In [168]:
df = pd.DataFrame(scraped) 
df

In [166]:
df.to_csv('propertypro.csv')